In [17]:
from io import BytesIO

import cairo
import IPython.display

In [18]:
# adapted from https://lists.cairographics.org/archives/cairo/2010-April/019691.html
def helper_quadratic_to(ctx, x1, y1, x2, y2):
    x0, y0 = ctx.get_current_point()
    ctx.curve_to(2.0 / 3.0 * x1 + 1.0 / 3.0 * x0,
                 2.0 / 3.0 * y1 + 1.0 / 3.0 * y0,
                 2.0 / 3.0 * x1 + 1.0 / 3.0 * x2,
                 2.0 / 3.0 * y1 + 1.0 / 3.0 * y2,
                 y1, y2);

In [19]:
def display_image(render_func, xsize, ysize):
    svgio = BytesIO()
    with cairo.SVGSurface(svgio, xsize, ysize) as surface:
        context = cairo.Context(surface)
        render_func(context)
    return svgio

In [22]:
def draw_demo_image(context):
    # These lines are copied verbatim from the
    # pycairo page: https://pycairo.readthedocs.io
    x, y, x1, y1 = 0.1, 0.5, 0.4, 0.9
    x2, y2, x3, y3 = 0.6, 0.1, 0.9, 0.5
    context.scale(200, 200)
    context.set_line_width(0.04)
    context.move_to(x, y)
    context.curve_to(x1, y1, x2, y2, x3, y3)
    context.stroke()
    context.set_source_rgba(1, 0.2, 0.2, 0.6)
    context.set_line_width(0.02)
    context.move_to(x, y)
    context.line_to(x1, y1)
    context.move_to(x2, y2)
    context.line_to(x3, y3)
    context.stroke()
    # end of pycairo copy

In [24]:
IPython.display.SVG(data=draw_demo_image(draw_figure, 200, 200).getvalue())

TypeError: draw_demo_image() takes 1 positional argument but 3 were given